## Exercise 12b: TD3 with MPC-based Function Approximation for pointmass task

In this exercise, we implement Twin-Delayed DDPG (TD3) for MPC-based approximation.

### Enviroment: Pointmass task
Linear dynamics defined with $A$ and $B$
Goal: Push the pointmass to the origin as shown in the following figure
<img src="imgs/pointmass.png" alt="400" width="400"/>
The baseline stage cost (reward) is \
$L(s,a) = 9x^2 + 9y^2 + \dot{x}^2 + \dot{y}^2 + F_x^2 + F_y^2$\
with the transition dynamics as 
$ s_+ = A s+ B a + bias + \mathcal{N}(0, 0.05)\\
A = \begin{bmatrix}
1 & 0 & 0.1 & 0 \\
0 & 1 & 0 & 0.1 \\
0 & 0 & 0.9 & 0 \\
0 & 0 & 0 & 0.9
\end{bmatrix},\,
B = \begin{bmatrix}
0 & 0 \\
0 & 0 \\
0.1 & 0 \\
0 & 0.1
\end{bmatrix}\,, bias = \begin{bmatrix}
0 \\
0 \\
0 \\
0
\end{bmatrix}$

### Agent: MPC-based agent
An MPC-based agent with $\theta$ learnable parameters (represented with $P$ in the exercise code)    

#### MPC formulation
- Symbolic solver definition:
    1) 'nlp_solver': gives actions ($\pi$) and value function $V$
    2) 'q_solver': gives action-value function $Q$ \
    with sensitivities $\nabla_\theta Q$, $\nabla_\theta V$ and $\nabla_\theta \pi$

- Stage cost model: Quadratic cost formulation
$L_\theta(s,a) = (s-s_{\mathrm{ref}})^T W_\theta W_\theta^T (s-s_{\mathrm{ref}}) + a^T R_\theta R_\theta^T a \\
T_\theta(s) = (s-s_{\mathrm{ref}})^T tW_\theta tW_\theta^T (s-s_{\mathrm{ref}})$ 
where $W_\theta$, $R_\theta$ and $tW_\theta$ are upper-triangular matrices.

- Transition model: state space model
    1) 'pointmass_v0_model': fixed A, B and bias
    2) 'pointmass_v1_model': learnable A, B and bias

#### MPC function approximation
Wrapper over the symbolic solver definitions.
It delivers the numerical values for $Q_\theta$, $V_\theta$ and $\pi_\theta$ with sensitivities $\nabla_\theta Q$, $\nabla_\theta V$ and $\nabla_\theta \pi$

#### Learning module 
- Twin-Delayed DDPG (TD3)\
Actor-critic algorithm with MPC as actor and DNN-based Q function as critic. \
The update rules with the deterministic policy gradient equation is\
$\theta \leftarrow \theta + \alpha \nabla_\theta \pi_\theta(s) \nabla_a Q(s,a)$\
with $\alpha$ as the learning rate.
- RL update step \
$\Delta \theta = \arg\min_{\Delta \theta} 0.5 \|\Delta \theta\|^2 + \alpha dJ^T \Delta \theta \\
\qquad \mathrm{s.t.} -0.2 \leq \Delta \theta \leq 0.2 \\ 
\theta = \theta + \Delta \theta \\
dJ = \mathbb{E}[\delta \nabla_\theta Q(s,a)]$

### Replay Buffer
Data buffer to store all the transition information $(s, a, s_+, L(s,a))$ along with some additional information (e.g. primal-dual variables, solver solutions)

### Experimental Setup

In [1]:
# Basic setup
import numpy as np
import casadi as csd
import torch
import torch.nn.functional as F
import torch.optim as optim

import pointmass
from mpc_formulation import MPCformulation
from mpc import MPCfunapprox
from replay_buffer import BasicBuffer, ReplayBuffer

obs_dim, action_dim = 4,2
seed = 1
env_str = "PointMass"
env_params = {
        "start_loc": [1.0, 1.0, 0.0, 0.0],
        "init_std": 0.2,
        "step_std": 0.05,
        "stochasticity": 1.0
    }
agent_params= {
        "gamma": 0.95,
        "model_str": "pointmass_v1_model",
        "opt_params": {
            "cost_defn": ["fullW", "fullR"],
            "cost_wt": [[1.0, 1.0, 1.0, 1.0], [0.1, 0.1]],
            "horizon": 10
        },
        "eps": 0.25,
        "learning_params": {
            "load_pretrained_net": True,
            "tau": 0.02,
            "critic_lr": 3e-3,
            "lr": 3e-3,
            "tr": 0.2,
            "train_params": {
                "iterations": 100,
                "batch_size": 32
            },
            "constrained_updates": True
        }
    }
n_iterations = 50
n_trains = 1
n_evals = 0
n_steps = 50
max_len_buffer = 50

# Experiment init
env = getattr(pointmass, env_str)("PointMass", env_params, seed)
replay_buffer = ReplayBuffer(max_len_buffer, seed)

In [2]:
# Additional functions
def rollout_sample(env, agent, mode="train"):
    state, obs = env.reset()
    agent.reset(obs)
    rollout_return = 0
    rollout_buffer = BasicBuffer()

    for _ in range(n_steps):
        action, add_info = agent.act_forward(obs, mode=mode)
        next_state, next_obs, reward, _ = env.step(action)
        if mode == "train":
            rollout_buffer.push(
                state, obs, action, reward, next_state, next_obs, add_info
            )
        rollout_return += reward
        state = next_state.copy()
        obs = next_obs.copy()
    return rollout_return, rollout_buffer


### TD3 algorithm

In [3]:
from nn_function_approximator import QNetwork, copy_net_param, update_net_param


class TD3_learning:
    def __init__(self, ocp, agent_params, seed):
        # hyperparameters
        self.ocp = ocp
        self._parse_agent_params(**agent_params)
        self.update_step = 0
        self.delay_step = 2
        self.rng = np.random.default_rng(seed)
        torch.manual_seed(seed)

        # initialize q networks
        self.q_net1 = QNetwork(self.ocp.obs_dim, self.ocp.action_dim)
        self.q_net2 = QNetwork(self.ocp.obs_dim, self.ocp.action_dim)
        self.target_q_net1 = QNetwork(self.ocp.obs_dim, self.ocp.action_dim)
        self.target_q_net2 = QNetwork(self.ocp.obs_dim, self.ocp.action_dim)

        # copy params to target param
        copy_net_param(self.target_q_net1, self.q_net1)
        copy_net_param(self.target_q_net2, self.q_net2)

        # initialize optimizers
        self.q1_optimizer = optim.Adam(self.q_net1.parameters(), lr=self.critic_lr)
        self.q2_optimizer = optim.Adam(self.q_net2.parameters(), lr=self.critic_lr)

    def get_action_batch(self, state_list, infos=None, act_wt=None, mode="update"):
        act_list = []
        for j, state in enumerate(state_list):
            soln = infos[j]["soln"] if infos is not None else None
            act, _ = self.ocp.act_forward(state, soln=soln, p_val=act_wt, mode=mode)
            act_list.append(act)
        return act_list

    def train(self, replay_buffer: ReplayBuffer):
        batch_size = min(self.batch_size, replay_buffer.size)
        train_it = min(self.iterations, int(5.0 * replay_buffer.size / batch_size))

        for _ in range(train_it):
            (
                _,
                obss,
                actions,
                rewards,
                _,
                next_obss,
                infos,
            ) = replay_buffer.sample(batch_size)
            obss = torch.FloatTensor(np.array(obss))
            actions = torch.FloatTensor(np.array(actions))
            rewards = torch.FloatTensor(np.array(rewards))
            next_obss = torch.FloatTensor(np.array(next_obss))

            # q loss
            next_actions = self.get_action_batch(next_obss, infos=infos)
            next_actions = torch.FloatTensor(np.array(next_actions))
            next_q1 = self.target_q_net1(next_obss, next_actions)
            next_q2 = self.target_q_net2(next_obss, next_actions)
            expected_q = rewards + self.ocp.gamma * torch.max(
                next_q1, next_q2
            )
            curr_q1 = self.q_net1.forward(obss, actions)
            curr_q2 = self.q_net2.forward(obss, actions)
            q1_loss = F.mse_loss(curr_q1, expected_q.detach())
            q2_loss = F.mse_loss(curr_q2, expected_q.detach())

            # update q networks
            self.q1_optimizer.zero_grad()
            q1_loss.backward()
            self.q1_optimizer.step()
            
            self.q2_optimizer.zero_grad()
            q2_loss.backward()
            self.q2_optimizer.step()

            # delayed update for actor and target q networks
            if self.update_step % self.delay_step == 0:
                dJ = 0.0
                for j, obs in enumerate(obss):
                    new_action, info = self.ocp.act_forward(
                        obs, soln=infos[j]["soln"], mode="update"
                    )
                    soln, pf_val, p_val = info["soln"], info["pf"], info["p"]
                    dPidP = self.ocp.dPidP(soln, pf_val, p_val)
                    
                    new_action += self.rng.normal(scale=0.1, size=(self.ocp.action_dim)).clip(-0.2,0.2)
                    new_action = new_action.clip(
                        self.ocp.model.action_space.low, 
                        self.ocp.model.action_space.high
                    )

                    new_action = torch.FloatTensor(new_action[None, :])
                    new_action.requires_grad_()
                    temp_q1 = self.q_net1.forward(obs[None, :], new_action)
                    temp_q1.backward()
                    grad_na = new_action.grad.numpy()
                    dJ += np.matmul(dPidP.T, grad_na.T)

                # Param update
                self.ocp.param_update(
                    dJ / batch_size,
                    constrained_updates=self.constrained_updates,
                )

                # target networks
                update_net_param(self.target_q_net1, self.q_net1, self.tau)
                update_net_param(self.target_q_net2, self.q_net2, self.tau)
            self.update_step += 1
        return {"l_theta": self.ocp.p_val}

    def _parse_agent_params(
        self,
        load_pretrained_net,
        tau,
        critic_lr,
        lr,
        tr,
        train_params,
        constrained_updates=False,
    ):
        self.load_pretrained_net = load_pretrained_net
        self.tau = tau
        self.critic_lr = critic_lr
        self.lr = lr
        self.tr = tr
        self.iterations = train_params["iterations"]
        self.batch_size = train_params["batch_size"]
        self.constrained_updates = constrained_updates


### MPC agent

In [4]:
class MPCfunapprox_ex(MPCfunapprox, MPCformulation):
    def __init__(self, agent_params, seed=1):
        # Parameters
        self._parse_agent_params(**agent_params)
        
        # Model init
        self.model = getattr(pointmass, self.model_str)()
        self.obs_dim = self.model.obs_space.shape[0]
        self.action_dim = self.model.action_space.shape[0]

        # MPC initilizaiton
        MPCformulation.__init__(self, self.model, self.opt_params, self.gamma)

        # Seeded random number generators
        self.rng1 = np.random.default_rng(seed)
        self.rng2 = np.random.default_rng(seed)
        self.rng3 = np.random.default_rng(seed)

        # Parameter values
        self.pf_val = 0.1 * self.rng1.random((self.nPf, 1))
        self.p_val = 0.1 * self.rng1.random((self.nP, 1))
        if "cost_wt" in self.opt_params:
            self.p_val[
                self.iP_cost[0] : self.iP_cost[1]
            ] = self.cost_model.cost_param_init(self.opt_params["cost_wt"])
        self.pf_val[self.iP_cost[1]:], self.p_val[self.iP_cost[1]:] = self.model.model_param_init() 
        
        # initiate the learning module
        self.learning_module = TD3_learning(self, self.learning_params, seed)
        self.constraint_param_opt(self.learning_module.lr, self.learning_module.tr) 

# Agent init
agent = MPCfunapprox_ex(agent_params, seed)
# Test run
_, obs = env.reset()
agent.reset(obs)
act0, info = agent.act_forward(obs)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



### Main Loop

In [5]:
# main loop
for it in range(n_iterations):
    print(f"Iteration: {it}")
    t_returns, e_returns = [],[]

    # training rollouts
    for _ in range(n_trains):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="train")
        replay_buffer.push(rollout_buffer.buffer)
        t_returns.append(rollout_return)

    # agent training
    agent.train(replay_buffer)

    # training rollouts
    for _ in range(n_evals):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="eval")
        e_returns.append(rollout_return)

    print(f"Training rollout return: {np.mean(t_returns)}")
    # print(f"Evaluation rollout return: {np.mean(e_returns)}")


Iteration: 0
W:
[[ 1.00005069e+00 -2.45183287e-04  1.95409572e-05 -4.94567247e-04]
 [-2.45183287e-04  9.99190661e-01  1.22576484e-04 -3.04615402e-04]
 [ 1.95409572e-05  1.22576484e-04  9.99920367e-01  2.57831068e-04]
 [-4.94567247e-04 -3.04615402e-04  2.57831068e-04  1.00076378e+00]]
R:
[[1.00250799e-02 5.87433668e-06]
 [5.87433668e-06 1.01137251e-02]]
Training rollout return: 204.9340106970251
Iteration: 1
W:
[[ 9.98521689e-01 -4.94305733e-03 -6.38785243e-04 -8.64691732e-03]
 [-4.94305733e-03  9.80147116e-01  7.66974468e-04 -7.69614662e-03]
 [-6.38785243e-04  7.66974468e-04  1.00114086e+00  5.30887274e-03]
 [-8.64691732e-03 -7.69614662e-03  5.30887274e-03  1.01984221e+00]]
R:
[[0.01060368 0.00023279]
 [0.00023279 0.01167534]]
Training rollout return: 296.74801213446705
Iteration: 2


CasADi - 2023-10-12 00:08:23 WARNING("KeyboardInterruptException") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:155]


SystemError: <built-in function Function_call> returned a result with an exception set

In [ ]:
# final rollouts
f_returns = []
for _ in range(100):
    rollout_return, rollout_buffer = rollout_sample(env, agent, mode="final")
    f_returns.append(rollout_return)
print(f"Final rollout return: {np.mean(f_returns)}")